# Preprocessing of CITES Data

#### Load Data Set

In [588]:
# Load required packages
import pandas as pd
import numpy as np

This data set has been downloaded from the Convention on International Trade in Endangered Species of Wild Fauna and Flora (CITES) website, available through https://trade.cites.org/, on 31 May 2024. 

In [622]:
# Import original data set
CITES_full = pd.read_csv("CITES31May.csv", sep = ";")

In [623]:
# Confirm import
CITES_full.head(3)

,Year,App.,Taxon,Class,Order,Family,Genus,Importer,Exporter,Origin,Importer reported quantity,Exporter reported quantity,Term,Unit,Purpose,Source
0,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,FR,GB,XX,NaN,4.0,skins,NaN,NaN,NaN
1,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,US,ES,XX,215.0,NaN,shoes,NaN,NaN,NaN
2,1977,II,Manis javanica,Mammalia,Pholidota,Manidae,Manis,US,FR,ID,NaN,444.0,skins,NaN,NaN,NaN


Check that the DataFrame has been loaded correctly, i.e. with 16 named columns from `Year` to `Source`.

#### Remove Unnecessary Variables

Several columns in the original data set will not be of use for this project, so we remove them now.

In [624]:
# Remove unnecessary variables from CITES data
CITES = CITES_full.drop(['App.', 'Class', 'Order', 'Family', 'Genus', 'Purpose', 'Source'], axis = 1)

#### Select African and Asian Exporters

Pangolins only occur naturally in Africa and certain parts of Asia (Rotich, 2018), and are incredibly difficult to breed in captivity (Environmental Investigation Agency, 2024).  We make the assumption that all `Exporter` countries that are not in regions where pangolins occur naturally have imported their stock from elsewhere, and hence we remove these transactions to avoid double-counting the items traded.  See Appendix A - Country Codes for a list of country codes and their regions.

In [625]:
# Create lists of African and Asian countries
African_countries = ["ZA", "TG", "CG", "CF", "CM", "LR", "NG", "TZ", "BJ", "DJ", "CI", "GN", "GQ", 
                     "GA", "CD", "KE", "GH", "UG", "BI", "BW", "SZ", "SN", "MZ"]
Asian_countries = ["CN", "JP", "TH", "SG", "ID", "MY", "TW", "PH", "LA", "HK", "KR", "MO", "VN", 
                   "KH", "PK", "LK"]

In [626]:
# Remove entries for non-African and non-Asian countries, but keep those with no recorded exporter
CITES = CITES[CITES['Exporter'].isin(African_countries + Asian_countries + ['XX', np.nan])]

If you are using this code for a data set more recent than the one provided for this project, you may need to update the lists of African and Asian countries based on what appears in the `Exporter` column.  Refer to the article by V. Rotich for more information on which countries contains wild pangolin populations.

#### Remove Double-Counted Transactions

According to the CITES Database Guidelines for use (CITES Secretariat and UNEP-WCMC, 2022), there may be cases where one transaction is recorded twice in the data set - once using the importer's report and once using the exporter's report.  We must remove these instances to avoid counting the same transactions twice.

In [627]:
# Create index for which rows to remove
repeat_index = []

# Loop to find obvious
for i in range(len(CITES)-1):
    if (CITES['Importer'].iloc[i] == CITES['Importer'].iloc[i+1]) & (CITES['Exporter'].iloc[i] == 
                                                                     CITES['Exporter'].iloc[i+1]):
        if (CITES['Importer reported quantity'].iloc[i] == CITES['Exporter reported quantity'].iloc[i+1]) | (
            CITES['Exporter reported quantity'].iloc[i] == CITES['Importer reported quantity'].iloc[i+1]):
            repeat_index.append(i)

# Loop to find resales
for i in range(len(CITES)-1):
    if (CITES['Exporter'].iloc[i] == CITES['Importer'].iloc[i+1]) & (CITES['Origin'].iloc[i] == 
                                                                     CITES['Exporter'].iloc[i+1]):
        if (CITES['Importer reported quantity'].iloc[i] == CITES['Exporter reported quantity'].iloc[i+1]) | (
            CITES['Exporter reported quantity'].iloc[i] == CITES['Importer reported quantity'].iloc[i+1]):
            repeat_index.append(i)

I further inspected the data and found a few anomalies.  There is no way to code to find these so I will remove them manually.

In [629]:
# Add the manual indices to repeat_index
repeat_index = repeat_index + [379, 412, 451, 485, 487, 493, 593, 663, 1014, 1029, 1073, 1095, 1130]
repeat_index = sorted(repeat_index)

In [630]:
# Remove the selected rows from the dataframe
CITES.drop(CITES.index[repeat_index], inplace=True)

Finally, we combine importer and exporter quantities to get one `Quantity` value per transaction.  According to the CITES Guidelines (CITES Secretariat and UNEP-WCMC, 2022), the exporter reported quantity is generally more reliable, so we will use that value where it is available.  Otherwise, we will use the importer reported quantity.

In [598]:
# Copy Exporter reported quantity for Quantity
CITES['Quantity'] = CITES['Exporter reported quantity']
quan_ind = CITES.columns.get_loc('Quantity')

# Fill in NaN Quantity values with Importer reported quantity
for i in range(len(CITES)):
    if np.isnan(CITES.iloc[i,quan_ind]):
        CITES.iloc[i,quan_ind] = CITES['Importer reported quantity'].iloc[i]

In [599]:
# Remove the Exporter and Importer reported quantity columns
CITES = CITES.drop(['Exporter reported quantity', 'Importer reported quantity'], axis = 1)

#### Indicator for African or Asian Species

Some entries in the data set list the `Taxon` as "Manidae spp." or "Manis spp.".  These are general terms, so in this case we will assign the pangolins to the region of their `Origin` if it is listed, and if not then we assign it to the `Exporter`'s region.

In [600]:
# Lists of species per region
African_species = ["Manis tetradactyla", "Manis tricuspis", "Manis gigantea", "Manis temminckii"]
Asian_species = ["Manis crassicaudata", "Manis javanica", "Manis culionensis", "Manis pentadactyla"]

In [601]:
# Create empty variable for Region
CITES['Region'] = ['Other']*len(CITES)
reg_ind = CITES.columns.get_loc('Region')

# Loop through to assign Region
for i in range(len(CITES)):
    if CITES['Taxon'].iloc[i] in African_species: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Taxon'].iloc[i] in Asian_species: 
        CITES.iloc[i,reg_ind] = "Asian"
    elif CITES['Origin'].iloc[i] in African_countries: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Origin'].iloc[i] in Asian_countries: 
        CITES.iloc[i,reg_ind] = "Asian"
    elif CITES['Exporter'].iloc[i] in African_countries: 
        CITES.iloc[i,reg_ind] = "African"
    elif CITES['Exporter'].iloc[i] in Asian_countries: 
        CITES.iloc[i,reg_ind] = "Asian"

# Convert to factor
CITES['Region'] = CITES['Region'].astype('category')

#### Calculate Number of Pangolins per Transaction

Each row in the data set lists a `Term`, detailing the type of product sold (e.g. skins, meat, specimens) and a `Unit` (e.g. kg, number of specimens, cartons).  We now convert these to the equivalent number of pangolins required to produce each `Term`.  See Appenddix B - Term Classification and Conversion for more details.

In [631]:
# Set up a list for all the terms where 1 unit equates to 1 pangolin
direct_equiv = ["bodies", "carvings", "claws", "leather", "leather items", "leather products (large)", 
                "live", "shoes", "skeletons", "skulls", "tails", "trophies"]

# Create data frame with species and weights
weights = {'Species': ["Manis gigantea", "Manis crassicaudata", "Manis temminckii", "Manis javanica", 
                       "Manis pentadactyla", "Manis tetradactyla", "Manis culionensis", 
                       "Manis tricuspis", "Manis spp.", "Manidae spp."], 
           'Weight kg': [33, 13, 12, 4.9, 3.6, 2.7, 2.1, 1.5, 9.1, 9.1]}
weights_df = pd.DataFrame(weights)

In [603]:
# Create column in CITES for number of pangolins 
CITES['Number'] = [0.0]*len(CITES)
num_ind = CITES.columns.get_loc('Number')

In [604]:
# Fill in the Number column for the simpler conversions
for i in range(len(CITES)):
    # For all direct equivalent terms
    if CITES['Term'].iloc[i] in direct_equiv:
        CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]
    # For all 'feet'
    elif CITES['Term'].iloc[i] == 'feet':
        CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]/4
    # For all 'leather products (small)'
    elif CITES['Term'].iloc[i] == 'leather products (small)':
        if CITES['Unit'].iloc[i] == 'pairs':
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]
        else: CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]/2

Unfortunately, due to the specific choices for conversion of each type of `Term` and `Unit` we will have to handle many of the conversions one by one below.

In [605]:
# Create column in CITES for kg (to be removed once we have populated the Number column)
CITES['kg'] = [0.0]*len(CITES)
kg_ind = CITES.columns.get_loc('kg')

# Find kg values for each entry
for i in range(len(CITES)):
    for j in range(len(weights_df)):
        if CITES['Taxon'].iloc[i] == weights_df['Species'].iloc[j]:
            CITES.iloc[i,kg_ind] = CITES['Quantity'].iloc[i]/weights_df['Weight kg'].iloc[j]

In [606]:
# Conversion for derivatives and powder
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] in ['derivatives', 'powder']:
        # For units in grams
        if CITES['Unit'].iloc[i] == 'g':
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/1000
        # For units in cartons or boxes
        elif (CITES['Unit'].iloc[i] == 'cartons') | (CITES['Unit'].iloc[i] == 'boxes'):
            CITES.iloc[i,num_ind] = 20*CITES['kg'].iloc[i]
        # For weight in kg and all other unspecified
        else: CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]           

In [607]:
# Conversion for meat
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] == 'meat':
        # For 'Number of specimens' unit
        if CITES['Unit'].iloc[i] == 'Number of specimens':
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]
        # For weights in grams
        elif CITES['Unit'].iloc[i] == 'g':
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/300
        # For weight in kg and all other unspecified
        else: CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/0.3

In [608]:
# Conversion for medicine
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] == 'medicine':
        # For units in g or ml
        if CITES['Unit'].iloc[i] in ['g', 'ml']:
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/1000
        # Otherwise
        else: CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]

In [609]:
# Conversion for scales and skin pieces
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] in ['scales', 'skin pieces']:
        # For units in g
        if CITES['Unit'].iloc[i] == 'g':
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/200
        # For units in m
        elif CITES['Unit'].iloc[i] == 'm':
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]/1.2
        # For cartons
        elif CITES['Unit'].iloc[i] == 'cartons':
            CITES.iloc[i,num_ind] = 100*CITES['kg'].iloc[i]
        # For number of specimens
        elif CITES['Unit'].iloc[i] == 'Number of specimens':
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]
        # For weights in kg and other unspecified
        else: CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/0.2        

In [610]:
# Conversion for skins
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] == 'skins':
        # For weights in kg
        if CITES['Unit'].iloc[i] == 'kg':
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/0.2
        # For m or m2
        elif CITES['Unit'].iloc[i] in ['m', 'm2']:
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]/1.2
        # For cm 
        elif CITES['Unit'].iloc[i] == 'cm':
            CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]/120
        # Otherwise
        else: CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]

In [611]:
# Conversion for specimens and unspecified
for i in range(len(CITES)):
    if CITES['Term'].iloc[i] in ['specimens', 'unspecified']:
        # For weights in kg
        if CITES['Unit'].iloc[i] == 'kg':
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]
        # For weights in g or ml
        elif CITES['Unit'].iloc[i] in ['g', 'ml']:
            CITES.iloc[i,num_ind] = CITES['kg'].iloc[i]/1000
        # For everything else
        else: CITES.iloc[i,num_ind] = CITES['Quantity'].iloc[i]

Now that we have all the conversions, we make one final assumption, which is that one cannot kill a fraction of a pangolin. We therefore will round all `Number` values that are not whole numbers up to the nearest whole number.  We must remove the 'kg' variable as it is no longer needed.  

In [612]:
# Round up all Number values
CITES['Number'] = np.ceil(CITES['Number'])

# Remove 'kg' column
CITES = CITES.drop(['kg'], axis = 1) 

#### Grouping of Trade Purposes

There are many different types of `Term` items listed, so I would like to group similar ones together to simplify the analysis.  For example, "leather" and "leather items" could be joined together into one group.

In [632]:
# Create lists for groupings
groupnames = ["whole", "trophy", "skins", "consumption", "derivatives"]
whole = ["bodies", "live", "specimens"]
trophy = ["trophies", "carvings", "shoes"]
skins = ["fur product (small)", "leather", "leather items", "leather products (large)", 
         "leather products (small)", "scales", "skin pieces", "skins"]
consumption = ["meat"]
derivatives = ["derivatives", "feet", "claws", "skeletons", "skulls", "tails", "unspecified"]
all_groups = [whole, trophy, skins, consumption, derivatives]

In [614]:
# Create new column in CITES to store the group name
CITES['Purpose'] = ['h']*len(CITES)
pur_ind = CITES.columns.get_loc('Purpose')

In [615]:
# Loop through all Terms to group them according to purpose
for i in range(len(CITES)):
    for j in range(len(groupnames)):
        if CITES['Term'].iloc[i] in all_groups[j]:
            CITES.iloc[i, pur_ind] = groupnames[j]

# Convert to categorical variable
CITES['Purpose'] = CITES['Purpose'].astype('category')

In [616]:
# Final check
CITES.head()

,Year,Taxon,Importer,Exporter,Origin,Term,Unit,Quantity,Region,Number,Purpose
3,1977,Manis pentadactyla,AU,CN,NaN,live,NaN,2.0,Asian,2.0,whole
4,1978,Manis crassicaudata,US,JP,XX,skins,NaN,298.0,Asian,298.0,skins
5,1978,Manis javanica,US,JP,XX,skins,NaN,1016.0,Asian,1016.0,skins
6,1978,Manis javanica,US,TH,NaN,live,NaN,1.0,Asian,1.0,whole
7,1978,Manis pentadactyla,AU,SG,NaN,leather products (small),NaN,1.0,Asian,1.0,skins


Finally we have our 'tidied' data set, which contains columns indicating the type, unit, and quantity of pangolin products traded; the importer, exporter, and origin; the species and region of origin of each pangolin; the number of whole pangolins involved per trade; and a broad grouping of different purposes for pangolin products.  We are now ready to export this data set and begin our analysis.

In [617]:
# Export the tidied data set
CITES.to_csv('CITES.csv')

#### References

1. CITES Secretariat and UNEP-WCMC (2022). A guide to using the CITES Trade Database. 
Version 9. Geneva, Switzerland, and Cambridge, UK.
2. Environmental Investigation Agency. (2024). Saving Pangolins from Extinction. Retrieved from Environmental Investigation Agency: https://eia-international.org/wildlife/helping-pangolins/saving-pangolins-from-extinction/
3. Rotich, V. (2018, September 04). Where Do Pangolins Live. Retrieved from WorldAtlas: https://www.worldatlas.com/articles/what-is-a-pangolin.html